## ¿Sirve o no sirve buscar precios de propiedades en campo description a traves de regex?
---

### Importo, preparo y cargo df

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_propierati = pd.read_csv('../properati.zip', compression='zip')
df_prop_uz = pd.read_csv('../properati.csv')

In [3]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 100)
#pd.set_option('precision', 5)

### Preguntas iniciales

Nulos en descripcion. Muy pocos. Parece prometedor.

In [4]:
df_propierati.description.isnull().value_counts()

False    121218
True          2
Name: description, dtype: int64

Extraigo valores de descripcion para buscar en regex101
(patron de busqueda ejemplo en: https://regex101.com/r/EhnQBv/1)

In [10]:
#salida = df_propierati.description.tail(100)
#salida.to_csv('descripcion_sample.txt', index=False)
#df_propierati.description.tail(100)

### Trabajo con expresiones regulares y mascaras booleanas.

1. Creo patron regex y lo compilo

 Patron: `([uU]\$?[sSdD\$]?[dD\$]?|dolares)\s?(\d+\.?\,?\d+\.?\,?\d+\.?\,?\d{1,}\,?\.?\d+)(?!(.*m2|.*mc|.*metros))`
- Comienza identificando la moneda (USD, usd, U\\$D, US\\$, etc.)
- Valor: desde 5 caracteres numericos, identifica comas (,) y puntos (.)
- Excluye al final caracteres que hagan referencia a metros cuadrados para evitar esos valores.

2. Creo objeto de regex en nueva columna matches_precio para idenficar las coincidencias del pattron en la columna description

3. Para identificar la efectividad y el valor de asignar precio desde la descripcion en el campo `price_aprox_usd` creo mascara booleana:

- valores nulos de columna `price_aprox_usd` 
- valores no nulos de columna `matches_precio`
- AND: mascara booleana con todos los valores nulos de `price_aprox_usd` que si tengan un valor identificado en el campo `matches_precio` (viene de `description`). __Resultado: 583 registros

In [36]:
# regex - key insensitive
precio_dolares_patt = '([uU]\$?[sSdD\$]?[dD\$]?|dolares)\s?(\d+\.?\,?\d+\.?\,?\d+\.?\,?\d{1,}\,?\.?\d+)(?!(.*m2|.*mc|.*metros))'
precio_dolares_regex = re.compile(precio_dolares_patt, flags = re.IGNORECASE)

# objetos regex en nueva columna
df_propierati['matches_precio'] = df_propierati.description.apply(lambda x: x if x is np.NaN else precio_dolares_regex.search(x))

# mascara booleana - valores nulos en price_aprox_usd que SI puede ser completados desde el regex sobre description
price_aprox_usd_mask = df_propierati.price_aprox_usd.isnull()
matches_precio_mask = df_propierati.matches_precio.notnull()
precio_desc_si_price_usd_no_mask = matches_precio_mask & price_aprox_usd_mask

precio_desc_si_price_usd_no_mask.value_counts()

# Comprobacion
# 583 registros
# df_propierati.loc[precio_desc_si_price_usd_no_mask, ['matches_precio','price_aprox_usd']].shape[0]

# Extraccion de muestra
df_propierati.loc[precio_desc_si_price_usd_no_mask, ['matches_precio','price_aprox_usd']].tail(5)

,matches_precio,price_aprox_usd
120910,"<re.Match object; span=(709, 718), match='u$s20.000'>",NaN
120911,"<re.Match object; span=(709, 718), match='u$s20.000'>",NaN
120912,"<re.Match object; span=(709, 718), match='u$s20.000'>",NaN
120913,"<re.Match object; span=(709, 718), match='u$s20.000'>",NaN
120914,"<re.Match object; span=(709, 718), match='u$s20.000'>",NaN


Falta depurar expresion regular:
- se estan agregando los precios de las cocheras.
- hay valores de precios no finales. Por ejemplo se observan valores de adelantos mas financiacion.

Ejemplos de valores repetidos:

In [40]:
df_propierati.description[120910:120914]

120910                    AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR ...
120911                    AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR ...
120912                    AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR ...
120913                    AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR ...
Name: description, dtype: object